In [1]:
from google.colab import files
uploaded = files.upload()

Saving DataTrain.csv to DataTrain.csv


In [2]:
pip install tldextract dnspython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.2 MB/s eta 0:00:00


In [11]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import time
import re
import socket
import tldextract
import dns.resolver
import math
from urllib.parse import urlparse
import numpy as np
import xgboost as xgb
from joblib import Parallel, delayed

In [16]:
import re
import math
import numpy as np
from urllib.parse import urlparse
import tldextract

def calculate_entropy(string):
    prob = [float(string.count(c)) / len(string) for c in set(string)]
    return -sum(p * math.log2(p) for p in prob)

def extract_features_from_url(url):
    features = {}
    parsed = urlparse(url)
    ext = tldextract.extract(url)

    domain = ext.registered_domain or ''
    subdomain = ext.subdomain or ''
    path = parsed.path or ''
    query = parsed.query or ''
    fragment = parsed.fragment or ''

    sub_parts = subdomain.split('.') if subdomain else []

    # 1. URL_based Features
    features['url_length'] = len(url)
    features['number_of_dots_in_url'] = url.count('.')
    features['having_repeated_digits_in_url'] = int(bool(re.search(r'(\d)\1', url)))
    features['number_of_digits_in_url'] = sum(c.isdigit() for c in url)
    features['number_of_special_char_in_url'] = len(re.findall(r'[^\w]', url))
    features['number_of_hyphens_in_url'] = url.count('-')
    features['number_of_underline_in_url'] = url.count('_')
    features['number_of_slash_in_url'] = url.count('/')
    features['number_of_questionmark_in_url'] = url.count('?')
    features['number_of_equal_in_url'] = url.count('=')
    features['number_of_at_in_url'] = url.count('@')
    features['number_of_dollar_in_url'] = url.count('$')
    features['number_of_exclamation_in_url'] = url.count('!')
    features['number_of_hashtag_in_url'] = url.count('#')
    features['number_of_percent_in_url'] = url.count('%')

    # 2. Domain_based Features
    features['domain_length'] = len(domain)
    features['number_of_dots_in_domain'] = domain.count('.')
    features['number_of_hyphens_in_domain'] = domain.count('-')
    features['having_special_characters_in_domain'] = int(bool(re.search(r'[^\w\.]', domain)))
    features['number_of_special_characters_in_domain'] = len(re.findall(r'[^\w\.]', domain))
    features['having_digits_in_domain'] = int(any(c.isdigit() for c in domain))
    features['number_of_digits_in_domain'] = sum(c.isdigit() for c in domain)
    features['having_repeated_digits_in_domain'] = int(bool(re.search(r'(\d)\1', domain)))

    # 3. Subdomain_based Features
    features['number_of_subdomains'] = len(sub_parts)
    features['having_dot_in_subdomain'] = int('.' in subdomain)
    features['having_hyphen_in_subdomain'] = int('-' in subdomain)
    features['average_subdomain_length'] = np.mean([len(p) for p in sub_parts]) if sub_parts else 0
    features['average_number_of_dots_in_subdomain'] = np.mean([s.count('.') for s in sub_parts]) if sub_parts else 0
    features['average_number_of_hyphens_in_subdomain'] = np.mean([s.count('-') for s in sub_parts]) if sub_parts else 0
    features['having_special_characters_in_subdomain'] = int(bool(re.search(r'[^\w\.]', subdomain)))
    features['number_of_special_characters_in_subdomain'] = len(re.findall(r'[^\w\.]', subdomain))
    features['having_digits_in_subdomain'] = int(any(c.isdigit() for c in subdomain))
    features['number_of_digits_in_subdomain'] = sum(c.isdigit() for c in subdomain)
    features['having_repeated_digits_in_subdomain'] = int(bool(re.search(r'(\d)\1', subdomain)))

    # 4. Other Features
    features['having_path'] = int(bool(path))
    features['path_length'] = len(path)
    features['having_query'] = int(bool(query))
    features['having_fragment'] = int(bool(fragment))
    features['having_anchor'] = int('#' in url)
    features['entropy_of_url'] = calculate_entropy(url)
    features['entropy_of_domain'] = calculate_entropy(domain)

    return features


In [ ]:
import math
import socket
import dns.resolver
from urllib.parse import urlparse
import tldextract
import re
import numpy as np

def calculate_entropy(string):
    """Tính entropy của chuỗi"""
    if not string:
        return 0
    prob = [float(string.count(c)) / len(string) for c in set(string)]
    return -sum(p * math.log2(p) for p in prob)

def get_dns_info(domain):
    """Trích xuất thông tin Nameserver"""
    try:
        ns_records = dns.resolver.resolve(domain, 'NS')
        ns_list = [str(r.target).strip('.') for r in ns_records]
    except:
        ns_list = []
    return ns_list

def get_ip_address(domain):
    """Trích xuất địa chỉ IP từ domain"""
    try:
        return socket.gethostbyname(domain)
    except:
        return None

def extract_features_from_url(url):
    features = {}

    parsed = urlparse(url)
    ext = tldextract.extract(url)

    domain = ext.registered_domain
    subdomain = ext.subdomain
    path = parsed.path or ''
    query = parsed.query or ''
    fragment = parsed.fragment or ''

    # ---- Feature extraction ----
    features['url_length'] = len(url)
    features['domain_length'] = len(domain)
    features['path_length'] = len(path)

    features['number_of_digits_in_url'] = sum(c.isdigit() for c in url)
    features['number_of_special_char_in_url'] = len(re.findall(r'[^\w]', url))
    features['number_of_dots_in_url'] = url.count('.')
    features['number_of_hyphens_in_url'] = url.count('-')
    features['number_of_underline_in_url'] = url.count('_')
    features['number_of_slash_in_url'] = url.count('/')
    features['number_of_questionmark_in_url'] = url.count('?')
    features['number_of_at_in_url'] = url.count('@')
    features['number_of_equal_in_url'] = url.count('=')
    features['number_of_percent_in_url'] = url.count('%')
    features['number_of_dollar_in_url'] = url.count('$')
    features['number_of_exclamation_in_url'] = url.count('!')
    features['number_of_hashtag_in_url'] = url.count('#')

    digits = re.findall(r'\d', url)
    features['having_repeated_digits_in_url'] = 1 if len(set(digits)) < len(digits) and len(digits) > 0 else 0

    # Domain
    features['number_of_digits_in_domain'] = sum(c.isdigit() for c in domain)
    features['number_of_hyphens_in_domain'] = domain.count('-')
    features['having_digits_in_domain'] = 1 if any(c.isdigit() for c in domain) else 0
    features['having_repeated_digits_in_domain'] = 1 if len(set(re.findall(r'\d', domain))) < len(re.findall(r'\d', domain)) and len(re.findall(r'\d', domain)) > 0 else 0
    features['having_special_characters_in_domain'] = 1 if re.search(r'[^\w\.]', domain) else 0
    features['number_of_special_characters_in_domain'] = len(re.findall(r'[^\w]', domain))

    # Subdomain
    subdomain_parts = subdomain.split('.') if subdomain else []
    features['number_of_subdomains'] = len(subdomain_parts)
    features['having_dot_in_subdomain'] = 1 if '.' in subdomain else 0
    features['having_hyphen_in_subdomain'] = 1 if '-' in subdomain else 0
    features['having_digits_in_subdomain'] = 1 if any(c.isdigit() for c in subdomain) else 0
    digits_in_sub = re.findall(r'\d', subdomain)
    features['having_repeated_digits_in_subdomain'] = 1 if len(set(digits_in_sub)) < len(digits_in_sub) and len(digits_in_sub) > 0 else 0
    features['number_of_digits_in_subdomain'] = len(digits_in_sub)
    features['number_of_special_characters_in_subdomain'] = len(re.findall(r'[^\w]', subdomain))
    features['having_special_characters_in_subdomain'] = 1 if re.search(r'[^\w\.]', subdomain) else 0

    features['average_subdomain_length'] = np.mean([len(p) for p in subdomain_parts]) if subdomain_parts else 0
    features['average_number_of_dots_in_subdomain'] = np.mean([s.count('.') for s in subdomain_parts]) if subdomain_parts else 0
    features['average_number_of_hyphens_in_subdomain'] = np.mean([s.count('-') for s in subdomain_parts]) if subdomain_parts else 0

    # Boolean flags
    features['having_path'] = 1 if path else 0
    features['having_query'] = 1 if query else 0
    features['having_fragment'] = 1 if fragment else 0
    features['having_anchor'] = 1 if '#' in url else 0

    # Entropy
    features['entropy_of_url'] = calculate_entropy(url)
    features['entropy_of_domain'] = calculate_entropy(domain)

    # Optional DNS/IP
    features['nameservers'] = get_dns_info(domain)
    features['ip_address'] = get_ip_address(domain)

    return features


In [14]:
df = pd.read_csv('DataTrain.csv')
df.head()

,url,label
0,https://www.southbankmosaics.com,0
1,https://www.uni-mainz.de,0
2,https://www.voicefmradio.co.uk,0
3,https://www.sfnmjournal.com,0
4,https://www.rewildingargentina.org,0


In [17]:
# Hàm extract_features_from_url đã loại bỏ DNS và IP
# Giả sử bạn đã có DataFrame df chứa cột 'url'

# Xử lý song song bằng tất cả CPU
results = Parallel(n_jobs=-1, backend="loky")(
    delayed(extract_features_from_url)(url) for url in df['url']
)

# Chuyển danh sách dict thành DataFrame
feature_df = pd.DataFrame(results)

# Gộp đặc trưng mới vào DataFrame gốc
df_with_features = pd.concat([df, feature_df], axis=1)

# Ghi ra file CSV
df_with_features.to_csv('DataTrain_with_features_2.csv', index=False)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DataTrain_with_features.csv to DataTrain_with_features (1).csv


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('DataTrain_with_features (1).csv')

In [ ]:
df.columns

Index(['url', 'label', 'url_length', 'domain_length', 'path_length',
       'number_of_digits_in_url', 'number_of_special_char_in_url',
       'number_of_dots_in_url', 'number_of_hyphens_in_url',
       'number_of_slash_in_url', 'number_of_questionmark_in_url',
       'number_of_at_in_url', 'number_of_equal_in_url',
       'number_of_percent_in_url', 'number_of_subdomains',
       'having_hyphen_in_subdomain', 'number_of_digits_in_subdomain',
       'having_special_characters_in_subdomain', 'number_of_digits_in_domain',
       'number_of_hyphens_in_domain', 'having_special_characters_in_domain',
       'entropy_of_url', 'entropy_of_domain', 'nameservers', 'ip_address'],
      dtype='object')

In [ ]:
X = df.iloc[:, 2:-2]
y = df['label']  # hoặc cột đúng chứa nhãn, nếu khác


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
X_train.head()

,url_length,domain_length,path_length,number_of_digits_in_url,number_of_special_char_in_url,number_of_dots_in_url,number_of_hyphens_in_url,number_of_slash_in_url,number_of_questionmark_in_url,number_of_at_in_url,...,number_of_percent_in_url,number_of_subdomains,having_hyphen_in_subdomain,number_of_digits_in_subdomain,having_special_characters_in_subdomain,number_of_digits_in_domain,number_of_hyphens_in_domain,having_special_characters_in_domain,entropy_of_url,entropy_of_domain
60279,39,21,10,1,7,1,0,4,0,0,...,0,0,0,0,0,0,0,0,4.221049,3.535175
21505,24,12,0,0,6,3,0,2,0,0,...,0,1,0,0,0,0,0,0,3.688722,3.084963
4639,18,6,0,2,5,2,0,2,0,0,...,0,1,0,0,0,2,0,0,3.350209,2.584963
49813,54,18,27,14,8,2,0,4,0,0,...,0,0,0,0,0,0,0,0,4.763966,3.530493
4484,37,25,0,0,5,2,0,2,0,0,...,0,1,0,0,0,0,0,0,4.208925,3.973661


In [ ]:
X_valid.head()

,url_length,domain_length,path_length,number_of_digits_in_url,number_of_special_char_in_url,number_of_dots_in_url,number_of_hyphens_in_url,number_of_slash_in_url,number_of_questionmark_in_url,number_of_at_in_url,...,number_of_percent_in_url,number_of_subdomains,having_hyphen_in_subdomain,number_of_digits_in_subdomain,having_special_characters_in_subdomain,number_of_digits_in_domain,number_of_hyphens_in_domain,having_special_characters_in_domain,entropy_of_url,entropy_of_domain
3419,32,20,0,0,6,2,1,2,0,0,...,0,1,0,0,0,0,1,1,3.858459,3.270951
62401,33,16,4,4,7,1,0,3,0,0,...,0,0,0,0,0,0,0,0,4.089552,3.327820
12987,25,13,0,0,5,2,0,2,0,0,...,0,1,0,0,0,0,0,0,3.893661,3.392747
51009,35,22,5,2,6,1,0,3,0,0,...,0,0,0,0,0,0,0,0,4.050013,3.516028
717,25,8,0,0,7,4,0,2,0,0,...,0,2,0,0,0,0,0,0,3.813661,2.750000


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Dự đoán
y_pred = rf.predict(X_valid)

# Độ chính xác
acc = accuracy_score(y_valid, y_pred)
print("Accuracy:", acc)

# Báo cáo chi tiết
print("\nClassification Report:")
print(classification_report(y_valid, y_pred))

# Ma trận nhầm lẫn
print("\nConfusion Matrix:")
print(confusion_matrix(y_valid, y_pred))

Accuracy: 0.9997700444580715

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       1.00      1.00      1.00      6523

    accuracy                           1.00     13046
   macro avg       1.00      1.00      1.00     13046
weighted avg       1.00      1.00      1.00     13046


Confusion Matrix:
[[6523    0]
 [   3 6520]]


In [ ]:
print(y_valid.value_counts())

label
0    6523
1    6523
Name: count, dtype: int64


In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# Tham số mẫu - thay bằng tham số bạn đã tối ưu được
params = {
    'n_estimators': 400,
    'max_depth': 10,
    'learning_rate': 0.1,
    'reg_lambda': 1,
    'reg_alpha': 0.5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
}

# Khởi tạo model
model = XGBClassifier(**params)

# Huấn luyện
model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
# Dự đoán
y_pred = rf.predict(X_valid)

# Độ chính xác
acc = accuracy_score(y_valid, y_pred)
print("Accuracy:", acc)

# Báo cáo chi tiết
print("\nClassification Report:")
print(classification_report(y_valid, y_pred))

# Ma trận nhầm lẫn
print("\nConfusion Matrix:")
print(confusion_matrix(y_valid, y_pred))

Accuracy: 0.9997700444580715

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       1.00      1.00      1.00      6523

    accuracy                           1.00     13046
   macro avg       1.00      1.00      1.00     13046
weighted avg       1.00      1.00      1.00     13046


Confusion Matrix:
[[6523    0]
 [   3 6520]]


In [ ]:
# Ví dụ 1 URL
features = extract_features_from_url("https://chatgpt.com/")

# Chuyển thành DataFrame và loại bỏ các cột không phải số
df_features = pd.DataFrame([features])
df_features = df_features.drop(columns=['nameservers', 'ip_address'])

In [ ]:
print(features)

{'url_length': 20, 'domain_length': 11, 'path_length': 1, 'number_of_digits_in_url': 0, 'number_of_special_char_in_url': 5, 'number_of_dots_in_url': 1, 'number_of_hyphens_in_url': 0, 'number_of_slash_in_url': 3, 'number_of_questionmark_in_url': 0, 'number_of_at_in_url': 0, 'number_of_equal_in_url': 0, 'number_of_percent_in_url': 0, 'number_of_subdomains': 0, 'having_hyphen_in_subdomain': 0, 'number_of_digits_in_subdomain': 0, 'having_special_characters_in_subdomain': 0, 'number_of_digits_in_domain': 0, 'number_of_hyphens_in_domain': 0, 'having_special_characters_in_domain': 0, 'entropy_of_url': 3.384183719779189, 'entropy_of_domain': 3.0957952550009344, 'nameservers': ['hassan.ns.cloudflare.com', 'savanna.ns.cloudflare.com'], 'ip_address': '172.64.155.209'}


In [ ]:
model_input_columns = X_train.columns.tolist()

In [ ]:
missing = set(model_input_columns) - set(df_features.columns)
print("Thiếu cột:", missing)

Thiếu cột: set()


In [ ]:
# Đảm bảo đúng thứ tự:
df_features = df_features[model_input_columns]

In [ ]:
prediction = rf.predict(df_features)
print(prediction)

[1]


In [ ]:
prediction = model.predict(df_features)
print(prediction)

[1]


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.csv to Dataset.csv


In [ ]:
df_test = pd.read_csv('Dataset.csv')
df_test.columns

Index(['Type', 'url_length', 'number_of_dots_in_url',
       'having_repeated_digits_in_url', 'number_of_digits_in_url',
       'number_of_special_char_in_url', 'number_of_hyphens_in_url',
       'number_of_underline_in_url', 'number_of_slash_in_url',
       'number_of_questionmark_in_url', 'number_of_equal_in_url',
       'number_of_at_in_url', 'number_of_dollar_in_url',
       'number_of_exclamation_in_url', 'number_of_hashtag_in_url',
       'number_of_percent_in_url', 'domain_length', 'number_of_dots_in_domain',
       'number_of_hyphens_in_domain', 'having_special_characters_in_domain',
       'number_of_special_characters_in_domain', 'having_digits_in_domain',
       'number_of_digits_in_domain', 'having_repeated_digits_in_domain',
       'number_of_subdomains', 'having_dot_in_subdomain',
       'having_hyphen_in_subdomain', 'average_subdomain_length',
       'average_number_of_dots_in_subdomain',
       'average_number_of_hyphens_in_subdomain',
       'having_special_characters_

In [ ]:
# Chỉ lấy các cột đặc trưng và nhãn từ cùng một DataFrame
selected_features = ['url_length', 'domain_length', 'path_length',
                     'number_of_digits_in_url', 'number_of_special_char_in_url',
                     'number_of_dots_in_url', 'number_of_hyphens_in_url',
                     'number_of_slash_in_url', 'number_of_questionmark_in_url',
                     'number_of_at_in_url', 'number_of_equal_in_url',
                     'number_of_percent_in_url', 'number_of_subdomains',
                     'having_hyphen_in_subdomain', 'number_of_digits_in_subdomain',
                     'having_special_characters_in_subdomain', 'number_of_digits_in_domain',
                     'number_of_hyphens_in_domain', 'having_special_characters_in_domain',
                     'entropy_of_url', 'entropy_of_domain']

# Cắt dữ liệu từ cùng một DataFrame (ví dụ: df_clean)
X_test = df_test[selected_features]
y_test = df_test['Type']

print(X_test.shape, y_test.shape)

(247950, 21) (247950,)


In [ ]:
X_train_test, X_valid_test, y_train_test, y_valid_test = train_test_split(
    X_test, y_test, test_size=0.2, stratify=y_test, random_state=42
)

In [ ]:
rf_test = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=1)

In [ ]:
rf_test.fit(X_train_test, y_train_test)

RandomForestClassifier(max_depth=10, n_estimators=500, random_state=1)

In [ ]:
y_pred_test = rf_test.predict(X_valid_test)

acc = accuracy_score(y_valid_test, y_pred_test)
print("Accuracy:", acc)

# Báo cáo chi tiết
print("\nClassification Report:")
print(classification_report(y_valid_test, y_pred_test))

# Ma trận nhầm lẫn
print("\nConfusion Matrix:")
print(confusion_matrix(y_valid_test, y_pred_test))

Accuracy: 0.8705182496471062

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.93      0.88     25708
           1       0.91      0.81      0.86     23882

    accuracy                           0.87     49590
   macro avg       0.88      0.87      0.87     49590
weighted avg       0.87      0.87      0.87     49590


Confusion Matrix:
[[23795  1913]
 [ 4508 19374]]


In [ ]:
# Ví dụ 1 URL
features = extract_features_from_url("https://www.facebook.com/")

# Chuyển thành DataFrame và loại bỏ các cột không phải số
df_features = pd.DataFrame([features])
df_features = df_features.drop(columns=['nameservers', 'ip_address'])
missing = set(model_input_columns) - set(df_features.columns)
print("Thiếu cột:", missing)
# Đảm bảo đúng thứ tự:
df_features = df_features[model_input_columns]
prediction = rf_test.predict(df_features)
print(prediction)

Thiếu cột: set()
[0]


In [ ]:
df_real = pd.read_csv('DataTrain_with_features (1).csv')

In [ ]:
X_real = df.iloc[:, 2:-2]
y_real = df['label']
print(X_real.shape, y_real.shape)

(65226, 21) (65226,)


In [ ]:
y_pred_real = rf_test.predict(X_real)

acc = accuracy_score(y_real, y_pred_real)
print("Accuracy:", acc)

# Báo cáo chi tiết
print("\nClassification Report:")
print(classification_report(y_real, y_pred_real))

# Ma trận nhầm lẫn
print("\nConfusion Matrix:")
print(confusion_matrix(y_real, y_pred_real))

Accuracy: 0.8862723453837427

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.81      0.88     32613
           1       0.83      0.97      0.89     32613

    accuracy                           0.89     65226
   macro avg       0.90      0.89      0.89     65226
weighted avg       0.90      0.89      0.89     65226


Confusion Matrix:
[[26308  6305]
 [ 1113 31500]]


In [ ]:
# Tham số mẫu - thay bằng tham số bạn đã tối ưu được
params = {
    'n_estimators': 296,
    'max_depth': 14,
    'learning_rate': 0.27734078865859557,
    'reg_lambda': 0.826885509199541,
    'reg_alpha': 0.2632655367006883,
    'subsampl388085e': 0.7444862040,
    'colsample_bytree': 0.8689700263924959,
    'random_state': 42,
}

# Khởi tạo model
xgb_test = XGBClassifier(**params)

# Huấn luyện
xgb_test.fit(X_train_test, y_train_test)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8689700263924959, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.27734078865859557,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=14, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=296, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred_test_xgb = xgb_test.predict(X_valid_test)

acc = accuracy_score(y_valid_test, y_pred_test_xgb)
print("Accuracy:", acc)

# Báo cáo chi tiết
print("\nClassification Report:")
print(classification_report(y_valid_test, y_pred_test_xgb))

# Ma trận nhầm lẫn
print("\nConfusion Matrix:")
print(confusion_matrix(y_valid_test, y_pred_test_xgb))

Accuracy: 0.9588425085702763

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     25708
           1       0.96      0.95      0.96     23882

    accuracy                           0.96     49590
   macro avg       0.96      0.96      0.96     49590
weighted avg       0.96      0.96      0.96     49590


Confusion Matrix:
[[24824   884]
 [ 1157 22725]]


In [ ]:
y_pred_real_xgb = xgb_test.predict(X_real)

acc = accuracy_score(y_real, y_pred_real_xgb)
print("Accuracy:", acc)

# Báo cáo chi tiết
print("\nClassification Report:")
print(classification_report(y_real, y_pred_real_xgb))

# Ma trận nhầm lẫn
print("\nConfusion Matrix:")
print(confusion_matrix(y_real, y_pred_real_xgb))

Accuracy: 0.8862723453837427

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.81      0.88     32613
           1       0.83      0.97      0.89     32613

    accuracy                           0.89     65226
   macro avg       0.90      0.89      0.89     65226
weighted avg       0.90      0.89      0.89     65226


Confusion Matrix:
[[26308  6305]
 [ 1113 31500]]


In [ ]:
# Ví dụ 1 URL
features = extract_features_from_url("https://emiratespass-online.com/mb/")

# Chuyển thành DataFrame và loại bỏ các cột không phải số
df_features = pd.DataFrame([features])
df_features = df_features.drop(columns=['nameservers', 'ip_address'])
missing = set(model_input_columns) - set(df_features.columns)
print("Thiếu cột:", missing)
# Đảm bảo đúng thứ tự:
df_features = df_features[model_input_columns]
prediction = xgb_test.predict(df_features)
if prediction[0] == 1:
    print("Phishing")
else:
    print("Legit")

Thiếu cột: set()
Phishing


In [ ]:
# Ví dụ 1 URL
features = extract_features_from_url("https://etcaer.xin/")

# Chuyển thành DataFrame và loại bỏ các cột không phải số
df_features = pd.DataFrame([features])
df_features = df_features.drop(columns=['nameservers', 'ip_address'])
missing = set(model_input_columns) - set(df_features.columns)
print("Thiếu cột:", missing)
# Đảm bảo đúng thứ tự:
df_features = df_features[model_input_columns]
prediction = xgb_test.predict(df_features)
if prediction[0] == 1:
    print("Phishing")
else:
    print("Legit")

Thiếu cột: set()
Phishing


In [ ]:
# Ví dụ 1 URL
features = extract_features_from_url("https://e-zpassny.com-ezoz.vip/")

# Chuyển thành DataFrame và loại bỏ các cột không phải số
df_features = pd.DataFrame([features])
df_features = df_features.drop(columns=['nameservers', 'ip_address'])
missing = set(model_input_columns) - set(df_features.columns)
print("Thiếu cột:", missing)
# Đảm bảo đúng thứ tự:
df_features = df_features[model_input_columns]
prediction = xgb_test.predict(df_features)
if prediction[0] == 1:
    print("Phishing")
else:
    print("Legit")

Thiếu cột: set()
Phishing


In [ ]:
# Ví dụ 1 URL
features = extract_features_from_url("https://www.facebook.com/")

# Chuyển thành DataFrame và loại bỏ các cột không phải số
df_features = pd.DataFrame([features])
df_features = df_features.drop(columns=['nameservers', 'ip_address'])
missing = set(model_input_columns) - set(df_features.columns)
print("Thiếu cột:", missing)
# Đảm bảo đúng thứ tự:
df_features = df_features[model_input_columns]
prediction = xgb_test.predict(df_features)
if prediction[0] == 1:
    print("Phishing")
else:
    print("Legit")

Thiếu cột: set()
Legit


In [ ]:
# Ví dụ 1 URL
features = extract_features_from_url("https://www.instagram.com/")

# Chuyển thành DataFrame và loại bỏ các cột không phải số
df_features = pd.DataFrame([features])
df_features = df_features.drop(columns=['nameservers', 'ip_address'])
missing = set(model_input_columns) - set(df_features.columns)
print("Thiếu cột:", missing)
# Đảm bảo đúng thứ tự:
df_features = df_features[model_input_columns]
prediction = xgb_test.predict(df_features)
if prediction[0] == 1:
    print("Phishing")
else:
    print("Legit")

Thiếu cột: set()
Legit


In [ ]:
# Ví dụ 1 URL
features = extract_features_from_url("https://www.youtube.com/")

# Chuyển thành DataFrame và loại bỏ các cột không phải số
df_features = pd.DataFrame([features])
df_features = df_features.drop(columns=['nameservers', 'ip_address'])
missing = set(model_input_columns) - set(df_features.columns)
print("Thiếu cột:", missing)
# Đảm bảo đúng thứ tự:
df_features = df_features[model_input_columns]
prediction = xgb_test.predict(df_features)
if prediction[0] == 1:
    print("Phishing")
else:
    print("Legit")

Thiếu cột: set()
Phishing


In [ ]:
# Ví dụ 1 URL
features = extract_features_from_url("https://ctt.hust.edu.vn/#")

# Chuyển thành DataFrame và loại bỏ các cột không phải số
df_features = pd.DataFrame([features])
df_features = df_features.drop(columns=['nameservers', 'ip_address'])
missing = set(model_input_columns) - set(df_features.columns)
print("Thiếu cột:", missing)
# Đảm bảo đúng thứ tự:
df_features = df_features[model_input_columns]
prediction = xgb_test.predict(df_features)
if prediction[0] == 1:
    print("Phishing")
else:
    print("Legit")

Thiếu cột: set()
Phishing


In [ ]:
Tối ưu tham số cho RF và so sánh performance metric, training time